In [5]:
# Install necessary libraries if not already installed
!pip install tensorflow
!pip install keras
!pip install numpy
!pip install pandas
!pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [6]:
!pip install -U scikit-learn --no-cache-dir

In [18]:
import requests
from bs4 import BeautifulSoup
import json
from time import sleep

# URLs for true and false statements
URLS = {
    "true": "https://www.politifact.com/factchecks/list/?ruling=true",
    "false": "https://www.politifact.com/factchecks/list/?ruling=false"
}

# Function to fetch data from a given URL
def fetch_statements(url, label, max_pages=5):
    data = []
    for page in range(1, max_pages + 1):
        # Update URL with page number
        paged_url = f"{url}&page={page}"
        response = requests.get(paged_url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Select statements on the page
        statements_divs = soup.select("div.m-statement__content")

        for div in statements_divs:
            # Extract statement, source, and ruling
            statement_element = div.select_one("div.m-statement__quote > a")
            source_element = div.select_one("a.m-statement__name")

            # Check if the elements exist before accessing 'text'
            statement = statement_element.text.strip() if statement_element else ""  # Use "" if statement_element is None
            source = source_element.text.strip() if source_element else ""  # Use "" if source_element is None

            # Append each record with label
            data.append({
                "statement": statement,
                "source": source,
                "truth": label
            })

        # To avoid overloading the server, we add a delay
        sleep(1)

    return data

# Fetch true and false statements
true_data = fetch_statements(URLS["true"], "True")
false_data = fetch_statements(URLS["false"], "False")

# Combine and save data
all_data = true_data + false_data
with open("politifact_data.json", "w") as f:
    json.dump(all_data, f, indent=4)

print("Data saved to politifact_data.json")

Data saved to politifact_data.json


In [19]:
import json
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the scraped data
with open('/content/politifact_data.json') as f:
    data = json.load(f)

# Extract statements and truth labels
statements = [item['statement'] for item in data]
labels = [item['truth'] for item in data]

# Convert "True" and "False" labels into binary format
binary_labels = ["fake" if label == "False" else "real" for label in labels]

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(binary_labels)  # 0 for 'real', 1 for 'fake'

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(statements)
sequences = tokenizer.texts_to_sequences(statements)
max_len = 100
X = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now you can proceed with model training as before



In [20]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# Define model parameters
vocab_size = 10000
embedding_dim = 64
lstm_units = 128

# Build the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Train the model
epochs = 5
batch_size = 64

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))


Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 516ms/step - accuracy: 0.4978 - loss: 0.6947 - val_accuracy: 0.4833 - val_loss: 0.6935
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 191ms/step - accuracy: 0.5153 - loss: 0.6938 - val_accuracy: 0.4833 - val_loss: 0.6945
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 200ms/step - accuracy: 0.4825 - loss: 0.6956 - val_accuracy: 0.4833 - val_loss: 0.6945
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 199ms/step - accuracy: 0.4828 - loss: 0.6928 - val_accuracy: 0.4833 - val_loss: 0.6939
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 193ms/step - accuracy: 0.5353 - loss: 0.6913 - val_accuracy: 0.4833 - val_loss: 0.6934


In [24]:
# Predict on new samples
new_statements = ["As of today, we have cut the flow of immigration by over half."]  # Replace with real examples
new_sequences = tokenizer.texts_to_sequences(new_statements)
new_padded = pad_sequences(new_sequences, maxlen=max_len, padding='post', truncating='post')
predictions = model.predict(new_padded)

# Convert predictions to labels
predicted_labels = ["fake" if p > 0.5 else "real" for p in predictions]
print(predicted_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
['fake']
